# **Pre-processing data : Kfold**
The aim of this jupyter notebook was creating a csv file that contains 10 different spilts of samples. Each of them must be at least one time a validation sample and training sample. The notebook has been created in the framework of an intership.\
This notebook has been inspired from 
Author : Julien Govoorts\
Organisation : ANAGEO ULB (Université libre de Bruxelles)

## Importing libraries

In [1]:
import numpy as np
import os, sys
import pandas as pd
import glob
import re
import natsort
import sklearn
import datetime
import cv2
import shutil
import time
import h5py

from osgeo import gdal
from osgeo import gdal_array
from natsort import natsorted
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# This function allows the jupyter notebook to find python modules in the SRC folder
import sys
sys.path.insert(0, "/home/jgovoort/SRC")

In [4]:
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that checks and creates folder
from mkdir import check_create_dir
# Import functions for plots
from plots import plot_loss, plot_pred_test, plot_pred_train, plot_loss_multirun
# Import functions for metrics to monitoring accuracy
from metrics import coeff_determination
# Import functions for display
from display import Norma_Xpercentile
# Import functions for exportation of results
from export_results import save_predictions, write_run_metrics_file
# Import LRFinder function
from LRFinder import LRFinder

## Importing data
This step import name of the different samples and its label (lulc). The data are store in a hdf5 file create in the YModel notebook from the chunckage module.

In [5]:
f = h5py.File('/home/jgovoort/Images/test_sample/tr_BNIR_patch/tr_Ymodel_S1BNIR.hdf5', 'r')
S1_patch_id = np.array(f["id_sar"])
S2_patch_id = np.array(f["id_rgb"])
S1_patch_lulc = np.array(f["lulc_sar"])
S2_patch_lulc = np.array(f["lulc_rgb"])
S1_patch_path = f["patch_sar_path"]
S2_patch_path = f["patch_rgb_path"]

In [6]:
lulc = S1_patch_lulc.reshape(S1_patch_lulc.size,1)
patch_id = S1_patch_id.reshape(S1_patch_lulc.size,1)

## Kfold : Dividing all the samples in 10 splits

In [7]:
kf_SEED = 101
kf = KFold(n_splits=5, random_state=kf_SEED, shuffle=True)
idx_train = []
idx_val = []
for train_index, val_index in kf.split(np.zeros(len(lulc)),lulc):
    idx_train.append(train_index)
    idx_val.append(val_index)
print(len(idx_train))
print([len(x) for x in idx_train])
print(len(idx_val))
print([len(x) for x in idx_val])

5
[3200, 3200, 3200, 3200, 3200]
5
[800, 800, 800, 800, 800]


In [8]:
kfold_nairobi_id = []
for i in range(len(idx_train)):
    kfold_nairobi_id.append((i+1,[x[0] for x in patch_id[idx_train[i]].tolist()],[x[0] for x in patch_id[idx_val[i]].tolist()]))

### Store the Kfold in a csv file

In [9]:
path_csv_kfold_split = '/home/jgovoort/Nairobi_10Fold_Split.csv'
a = pd.DataFrame(kfold_nairobi_id, columns=['kfold','ID_train','ID_val'])
a.to_csv(path_csv_kfold_split, index=False)
a

,kfold,ID_train,ID_val
0,1,"[110, 360, 614, 865, 1370, 1372, 1881, 2120, 2...","[104, 859, 1118, 1875, 3892, 4883, 5985, 6671,..."
1,2,"[104, 110, 360, 614, 859, 865, 1118, 1372, 187...","[1370, 4719, 4880, 4905, 5392, 5747, 5752, 591..."
2,3,"[104, 110, 614, 859, 865, 1118, 1370, 1372, 18...","[360, 3358, 4136, 4884, 5161, 5736, 6189, 6433..."
3,4,"[104, 110, 360, 614, 859, 1118, 1370, 1875, 18...","[865, 1372, 2129, 2882, 2884, 3124, 4145, 4623..."
4,5,"[104, 360, 859, 865, 1118, 1370, 1372, 1875, 2...","[110, 614, 1881, 2120, 3369, 4632, 5988, 6181,..."
